[Reference](https://medium.com/@obianuju.c.okafor/automatic-topic-classification-of-research-papers-using-the-nlp-topic-model-nmf-d4365987ec82)

![Step](https://miro.medium.com/max/1400/1*wKi39V-CUbtjr5cvW7a6tQ.png)

In [2]:
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
import re
import gensim
import gensim.corpora as corpora
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
!pip install pyLDAvis
import pyLDAvis
from pyLDAvis import sklearn as sklearn_lda

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
     |████████████████████████████████| 1.6MB 2.5MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=8adf24ef0d7e9433937310acfa7fae00005795a83ebe54ac1e2c304dea9442d5
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis


In [3]:
#Load data file
dataset = pd.read_csv(r'https://raw.githubusercontent.com/Branden-Kang/Python-practice/master/Data/research_papers.csv', encoding='ISO-8859–1')
dataset.head()

,Id,Reference,Authors,Title,Year,Conference/ Journal,Codes,Abstract,Conclusion
0,1,"Ludi, S., & Spencer, M. (2017). Design Consid...","Ludi, S., & Spencer, M.",Design Considerations to Increase Block-based ...,2017.0,Journal of Visual Languages and Sentient Sys...,"BBL, DES, SR, KN, CE, CN, DT, CC",Block-based programming languages are a popula...,The initial version of accessible Blockly shou...
1,2,"Ludi, S., Simpson, J., & Merchant, W. (2016, O...","Ludi, S., Simpson, J., & Merchant, W.",Exploration of the use of auditory cues in cod...,2016.0,ACM SIGACCESS Conference on Computers and Acce...,"AP, CN, CC, BBL, DES, DT, ACUE, SM",Visual programming languages are commonplace i...,NaN
2,3,"Ludi, S. (2015, October). Position paper: Towa...","Ludi, S.",Towards making block-based programming accessi...,2015.0,IEEE Blocks and Beyond Workshop. IEEE.,"BBL, DES, DT, WAPP, PC",Block-based programming environments are not a...,NaN
3,4,"Milne, L. R., & Ladner, R. E. (2018, April). B...","Milne, L. R., & Ladner, R. E.\r",Blocks4All: overcoming accessibility barriers ...,2018.0,CHI Conference on Human Factors in Computing S...,"BBL, CH, CN, CC, SR, TS, SM, KDG, MSH",Blocks-based programming environments are a po...,We conducted an evaluation of current blocks-b...
4,5,"Caraco, L. B., Deibel, S., Ma, Y., & Milne, L....","Caraco, L. B., Deibel, S., Ma, Y., & Milne, L....",Making the Blockly Library Accessible via Touc...,2019.0,In The 21st International ACM SIGACCESS Confer...,"BBL, TS, SR, DES, CN, CE",Block-based programming environments are a pop...,We present two interfaces which were designed ...


# Clean Data

In [4]:
#Remove the unecessary columns
dataset = dataset.drop(columns=['Id', 'Reference', 'Codes', 'Authors', 'Year', 'Conference/ Journal'], axis=1)
#Fill in the empty cells
dataset = dataset.fillna('No conclusion')
#Merge abstract and conclusion
dataset['Paper_Text'] = dataset["Abstract"] + dataset["Conclusion"]
#show first 5 records
dataset.head()

,Title,Abstract,Conclusion,Paper_Text
0,Design Considerations to Increase Block-based ...,Block-based programming languages are a popula...,The initial version of accessible Blockly shou...,Block-based programming languages are a popula...
1,Exploration of the use of auditory cues in cod...,Visual programming languages are commonplace i...,No conclusion,Visual programming languages are commonplace i...
2,Towards making block-based programming accessi...,Block-based programming environments are not a...,No conclusion,Block-based programming environments are not a...
3,Blocks4All: overcoming accessibility barriers ...,Blocks-based programming environments are a po...,We conducted an evaluation of current blocks-b...,Blocks-based programming environments are a po...
4,Making the Blockly Library Accessible via Touc...,Block-based programming environments are a pop...,We present two interfaces which were designed ...,Block-based programming environments are a pop...


# Preprocess Data

In [5]:
#function for lemmatization
def get_lemma(word):
  lemma = wn.morphy(word)
  if lemma is None:
    return word
  else:
    return lemma# tokenization
tokenized_data = dataset['Paper_Text'].apply(lambda x: x.split())# Remove punctuation
tokenized_data = tokenized_data.apply(lambda x: [re.sub('[-,()\\!?]', '', item) for item in x])
tokenized_data = tokenized_data.apply(lambda x: [re.sub('[.]', ' ', item) for item in x])# turn characters to lowercase
tokenized_data = tokenized_data.apply(lambda x: [item.lower() for item in x])# remove stop-words
stop_words = stopwords.words('english')
stop_words.extend(['from','use', 'using','uses','user', 'users', 'well', 'study', 'survey', 'think'])# remove words of length less than 3
tokenized_data = tokenized_data.apply(lambda x: [item for item in x if item not in stop_words and len(item)>3])# lemmatize by calling lemmatization function
tokenized_data= tokenized_data.apply(lambda x: [get_lemma(item) for item in x])

# Create Bigrams and Trigrams

In [7]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(tokenized_data, min_count=5, threshold=10) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[tokenized_data], threshold=10)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# Define functions for creating bigrams and trigrams.
def make_bigrams(texts):
  return [bigram_mod[doc] for doc in texts]
  
def make_trigrams(texts):
  return [trigram_mod[bigram_mod[doc]] for doc in texts]
  
# Form Bigrams
tokenized_data_bigrams = make_bigrams(tokenized_data)

# Form Trigrams
tokenized_data_trigrams = make_trigrams(tokenized_data)

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [8]:
# de-tokenization, combine tokens together
detokenized_data = []
for i in range(len(dataset)):
    t = ' '.join(tokenized_data_trigrams[i])
    detokenized_data.append(t)

dataset['clean_text']= detokenized_data
documents = dataset['clean_text']

# Create Document-Term Matrix

In [9]:
#Set variable number of terms
no_terms = 1000

# NMF uses the tf-idf count vectorizer
# Initialise the count vectorizer with the English stop words
vectorizer = TfidfVectorizer(max_df=0.5, min_df=2, max_features=no_terms, stop_words='english')

# Fit and transform the text
document_matrix = vectorizer.fit_transform(documents)

#get features
feature_names = vectorizer.get_feature_names()

# Generate topics using Topic Model

In [11]:
#Set variables umber of topics and top words.
no_topics = 10
no_top_words = 10

# Function for displaying topics
def display_topic(model, feature_names, num_topics, no_top_words, model_name):
    print("Model Result:")
    word_dict = {}
    for i in range(num_topics):
      #for each topic, obtain the largest values, and add the words they map to into the dictionary.
       words_ids = model.components_[i].argsort()[:-no_top_words - 1:-1]
       words = [feature_names[key] for key in words_ids]
       word_dict['Topic # ' + '{:02d}'.format(i)] = words    
       
    dict = pd.DataFrame(word_dict)
    dict.to_csv('%s.csv' % model_name)
    return dict

# Apply NMF topic model to document-term matrix
nmf_model = NMF(n_components=no_topics, random_state=42, alpha=.1, l1_ratio=.5, init='nndsvd').fit(document_matrix)

In [12]:
#display topics for nmf model
display_topic(nmf_model, feature_names, no_topics, no_top_words, 'NMF_Model_Result')

Model Result:


,Topic # 00,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09
0,program,child,blind_learner,blind_student,blockbased_programming,blind_developer,structjumper,story,sight_programmer,solution
1,auditory,learning,audio_programming_language,barrier,blockly,challenge,skimming,block,blind_programmer,blind
2,auditory_cue,torino,apl,computer_science,computing,developer,participant,accessible,blind,development
3,nonsighted,design,skill,project,environment,face,structure,blockbased_programming_language,area,aid
4,computer,computational,novice_blind_learner,structjumper,library,software_developer,blind_programmer,tangible,reading,description_language
5,research,inclusive,solve_problem,interview,impaired,software,complete,student,prioritize,graphic_interface
6,sound,physical,interact,information,design,difficulty,class,game,method,regard
7,blind,vision,motivating,create,visually,ides,eclipse,output,difference,usage
8,cue,teacher,help,goal,outreach,blind,navigate,audio,suggest,graphical_interface
9,comprehension,nonspecialist,write_program,dissertation,foster,need,nest,novice_programmer,screen_reader,problem


# Analyzing our NMF model

In [13]:
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = sklearn_lda.prepare(nmf_model, document_matrix, vectorizer)
LDAvis_prepared

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:223: RuntimeWarning: divide by zero encountered in log
  kernel = (topic_given_term * np.log((topic_given_term.T / topic_proportion).T))
/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:240: RuntimeWarning: divide by zero encountered in log
  log_lift = np.log(topic_term_dists / term_proportion)
/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:241: RuntimeWarning: divide by zero encountered in log
  log_ttd = np.log(topic_term_dists)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.046432 -0.060745       1        1  28.984868
1     -0.280743  0.107633       2        1  13.272847
7     -0.245475  0.038103       3        1  11.915174
5      0.160219  0.075452       4        1   9.645879
6      0.220662  0.129751       5        1   7.580157
3      0.031313  0.286639       6        1   6.457751
2     -0.100707 -0.039736       7        1   6.253687
4     -0.151655 -0.261759       8        1   6.173575
8      0.199473 -0.225725       9        1   5.141566
9      0.120481 -0.049613      10        1   4.574497, topic_info=                 Term      Freq     Total Category  logprob  loglift
106             child  3.000000  3.000000  Default  30.0000  30.0000
660      structjumper  3.000000  3.000000  Default  29.0000  29.0000
79      blind_student  2.000000  2.000000  Default  28.0000  28.0000
75              blind  4.000000  4.000000  Default  27.0000  27.0000
78   blind_programmer  2.000000  2.000000  Default  26.0000  26.0000
..                ...       ...       ...      ...      ...      ...
38           approach  0.289095  1.724984  Topic10  -4.0044   1.2985
427              make  0.169504  0.959269  Topic10  -4.5382   1.3514
584          research  0.179026  2.286990  Topic10  -4.4836   0.5372
191         different  0.164384  1.086168  Topic10  -4.5689   1.1965
230            enable  0.160276  1.217715  Topic10  -4.5942   1.0568

[563 rows x 6 columns], token_table=      Topic      Freq                         Term
term                                              
3         3  1.080517                    abilities
4         2  1.083351                      ability
8         8  0.819794                       access
9         1  0.544254                accessibility
10        3  0.757868                   accessible
...     ...       ...                          ...
745       2  0.955128                       vision
747       1  1.197471  visual_programming_language
749       8  1.563385                     visually
758       1  0.703409                         work
769       2  1.932203                        young

[175 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 8, 6, 7, 4, 3, 5, 9, 10])

# Classify papers under topics

In [14]:
#Use NMF model to assign topic to papers in corpus
nmf_topic_values = nmf_model.transform(document_matrix)
dataset['NMF Topic'] = nmf_topic_values.argmax(axis=1)

#Save dataframe to csv file
# dataset.to_csv('final_results.csv')
dataset.head(10)

,Title,Abstract,Conclusion,Paper_Text,clean_text,NMF Topic
0,Design Considerations to Increase Block-based ...,Block-based programming languages are a popula...,The initial version of accessible Blockly shou...,Block-based programming languages are a popula...,blockbased_programming_language popular means ...,4
1,Exploration of the use of auditory cues in cod...,Visual programming languages are commonplace i...,No conclusion,Visual programming languages are commonplace i...,visual_programming_language commonplace engage...,0
2,Towards making block-based programming accessi...,Block-based programming environments are not a...,No conclusion,Block-based programming environments are not a...,blockbased_programming environment accessible ...,4
3,Blocks4All: overcoming accessibility barriers ...,Blocks-based programming environments are a po...,We conducted an evaluation of current blocks-b...,Blocks-based programming environments are a po...,blocksbased_programming environment popular to...,1
4,Making the Blockly Library Accessible via Touc...,Block-based programming environments are a pop...,We present two interfaces which were designed ...,Block-based programming environments are a pop...,blockbased_programming environment popular lea...,4
5,Expanding Blocks4All with Variables and Funct...,Blocks-based programming environments are ofte...,Our enhancements to Blocks4All have improved i...,Blocks-based programming environments are ofte...,blocksbased_programming environment often inac...,1
6,An Accessible Blocks Language: Work in Progress.,Block languages are extensively used to introd...,No conclusion,Block languages are extensively used to introd...,block language extensively use introduce progr...,7
7,Nonvisual Visual Programming.,Visual programming systems are widely used to ...,"Nonvisual visual programming is possible, and ...",Visual programming systems are widely used to ...,visual programming system widely use introduce...,1
8,Accessible AST-Based Programming for Visually-...,Most programmers rely on visual tools (block-b...,While this evaluation focused exclusively on u...,Most programmers rely on visual tools (block-b...,programmer rely visual tool blockbased editor ...,6
9,Programming microworlds for visually impaired...,The paper describes our research aimed at veri...,Our environments are physical micro-worlds [11...,The paper describes our research aimed at veri...,paper_describe research aim verification suita...,1
